In [ ]:
pip install -q datasets --use-deprecated=legacy-resolver

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 7.1 MB/s eta 0:00:00
ERROR: pip's legacy dependency resolver does not consider dependency conflicts when selecting packages. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you'll have fsspec 2024.9.0 which is incompatible.


In [ ]:
import pandas as pd
from datasets import Dataset
df = pd.read_csv('/content/WELFake_Dataset.csv', nrows=10000, on_bad_lines='skip')

In [ ]:
df["content"] = df["text"]

# Keep only necessary columns
df = df[["content", "label"]]

# Convert to Hugging Face Dataset format
dataset = Dataset.from_pandas(df)

In [ ]:
dataset = dataset.train_test_split(test_size=0.2)
train_dataset = dataset["train"]
test_dataset = dataset["test"]

In [ ]:
train_dataset = train_dataset.filter(lambda x: x["content"] is not None)
test_dataset = test_dataset.filter(lambda x: x["content"] is not None)

Filter:   0%|          | 0/8000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [ ]:
from transformers import DistilBertTokenizer

# Initialize the tokenizer
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

# Define tokenization function
def tokenize(batch):
    return tokenizer(batch["content"], padding="max_length", truncation=True)

# Apply tokenization
train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

# Set format for PyTorch
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Map:   0%|          | 0/7993 [00:00<?, ? examples/s]

Map:   0%|          | 0/1999 [00:00<?, ? examples/s]

In [ ]:
from transformers import EarlyStoppingCallback
from accelerate import Accelerator
import torch

# Initialize Accelerator for TPU usage
accelerator = Accelerator()


In [ ]:
from transformers import DistilBertForSequenceClassification

# Load DistilBERT model
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",  # Directory where model checkpoints and logs will be saved
    evaluation_strategy="epoch",  # Evaluate the model at the end of each epoch
    # learning_rate=5e-4,  # Adjust learning rate
    per_device_train_batch_size=4,  # Small batch size for training (adjust based on your hardware)
    per_device_eval_batch_size=16,  # Larger batch size for evaluation
    gradient_accumulation_steps=4,  # Accumulate gradients over 4 steps to simulate larger batch sizes
    num_train_epochs=10,  # Number of training epochs
    weight_decay=0.01,  # Weight decay for regularization
    logging_dir="./logs",  # Directory for logging
    save_strategy="epoch",  # Save model at the end of each epoch
    save_total_limit=3,  # Keep only the last 3 saved models to avoid excessive disk usage
    metric_for_best_model="accuracy",  # Metric to determine the best model (if you're using accuracy)
    fp16=True,  # Enable mixed precision training (useful for faster training on GPUs)
    report_to="wandb",  # Optionally use WandB for tracking metrics during training
    dataloader_num_workers=2,  # Number of workers for data loading (can speed up loading on CPUs)
    load_best_model_at_end=True  # Ensures the best model is restored at the end of training
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
for param in model.distilbert.parameters():
    param.requires_grad = False

# for name, param in model.named_parameters():
#   print(f"{name}: {'Trainable' if param.requires_grad else 'Frozen'}")

In [ ]:
from transformers import Trainer, TrainerCallback, AdamW, get_linear_schedule_with_warmup
from transformers import EarlyStoppingCallback
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding

# Optimizer and scheduler setup
optimizer = AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)

total_steps = len(train_dataset) * training_args.num_train_epochs
warmup_steps = int(0.1 * total_steps)  # 10% of the total steps for warm-up

# Scheduler setup
lr_scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=total_steps)

# Data collator for padding (if using transformers)
data_collator = DataCollatorWithPadding(tokenizer)

# Early Stopping Callback
early_stopping = EarlyStoppingCallback(early_stopping_patience=2)  # Stop after 2 epochs without improvement

# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    callbacks=[early_stopping],
    data_collator=data_collator,  # Ensure correct data collator is used
    optimizers=(optimizer, lr_scheduler)  # Include optimizer and scheduler
)

# Train the model
trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
0,No log,0.683316,0.544772,0.688782,0.535069,0.966411
1,0.691100,0.647376,0.786893,0.817637,0.738022,0.916507
2,0.667900,0.583787,0.813907,0.829826,0.792832,0.870441
3,0.619200,0.509857,0.814407,0.824752,0.812093,0.837812
4,0.549500,0.446471,0.820910,0.827220,0.832039,0.822457
5,0.476200,0.403489,0.829915,0.833984,0.848907,0.819578
6,0.420000,0.370637,0.841921,0.845854,0.860119,0.832054
7,0.379500,0.349119,0.848424,0.850518,0.875127,0.827255
8,0.351400,0.326741,0.856928,0.860352,0.875746,0.845489
9,0.328300,0.318839,0.862931,0.862173,0.905920,0.822457


TrainOutput(global_step=4990, training_loss=0.47995638187997086, metrics={'train_runtime': 551.1676, 'train_samples_per_second': 145.019, 'train_steps_per_second': 9.054, 'total_flos': 1.057619710869504e+16, 'train_loss': 0.47995638187997086, 'epoch': 9.993996998499249})

In [ ]:
results = trainer.evaluate()
print(results)

{'eval_loss': 0.3188394010066986, 'eval_accuracy': 0.8629314657328664, 'eval_f1': 0.8621730382293763, 'eval_precision': 0.9059196617336153, 'eval_recall': 0.8224568138195777, 'eval_runtime': 7.9217, 'eval_samples_per_second': 252.344, 'eval_steps_per_second': 15.779, 'epoch': 9.993996998499249}


In [ ]:
# Activate trainning of all parameters
for param in model.distilbert.parameters():
    param.requires_grad = True

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
0,No log,0.107428,0.980490,0.981003,0.996044,0.966411
1,0.122900,0.121514,0.980990,0.981500,0.996047,0.967370
2,0.034600,0.090470,0.983992,0.984511,0.993164,0.976008
3,0.016100,0.114161,0.986493,0.987149,0.979226,0.995202
4,0.007900,0.137132,0.982491,0.983051,0.992180,0.974088
5,0.009300,0.118996,0.982991,0.983511,0.994118,0.973129


TrainOutput(global_step=2995, training_loss=0.03315858212058652, metrics={'train_runtime': 798.4707, 'train_samples_per_second': 100.104, 'train_steps_per_second': 6.249, 'total_flos': 6348897482784768.0, 'train_loss': 0.03315858212058652, 'epoch': 5.99799899949975})

In [ ]:
results = trainer.evaluate()
print(results)

{'eval_loss': 0.11416067183017731, 'eval_accuracy': 0.9864932466233116, 'eval_f1': 0.9871489766777725, 'eval_precision': 0.9792256846081209, 'eval_recall': 0.9952015355086372, 'eval_runtime': 7.8641, 'eval_samples_per_second': 254.194, 'eval_steps_per_second': 15.895, 'epoch': 5.99799899949975}


In [ ]:
save_directory = '/content/distilbert_model'
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)